In [1]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json
import webbrowser
import keyboard


path_to_model = '/Users/beene/Documents/BMM-2023/kostt/1.pth' #ПУТЬ ДО МОДЕЛИ
path_to_dataset = '/Users/beene/Documents/BMM-2023/kostt/dataset' #ПУТЬ ДО ДАТАСЕТА



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 5) #ВТОРОЙ АТРИБУТ - КОЛИЧЕСТВО ЖЕСТОВ
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x
    


net = torch.load(path_to_model) 
net.eval()


class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)
    
data, temp = [],[]


gests = os.listdir(path_to_dataset)


print('ЖЕСТЫ')
for gest in gests:
    data = []
    print(gest)
    f = open(path_to_dataset + '/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
print()
data = np.array(temp, dtype = 'object') #МОЖЕТ РУГАТЬСЯ НА МАССИВ, ХЗ ОТ ЧЕГО ЭТО ЗАВИСИТ, ЕСЛИ РУГАЕТСЯ - УДАЛИ dtype = 'object'
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

tmaxmass =[[0,0],[0,0],[0,0],[0,0],[0,0]] # ДЛИНА МАССИВА ДАЛЖНА БЫТЬ РАВНА КОЛИЧЕСТВУ ЖЕСТОВ
#tmaxmass = [[1,2],[3,4],[5,6],[7,8]]
for epoch in range(15):
    for samples, labels in dataloader:
        for j in samples:
            pred = net(j).tolist()
            mx = max(pred)
            mxid = pred.index(mx)
            tmaxmass[mxid][0] = tmaxmass[mxid][0] + mx
            tmaxmass[mxid][1] = tmaxmass[mxid][1] + 1
maxmass = [tmaxmass[i][0] / tmaxmass[i][1] for i in range(len(tmaxmass)) ]
print('СРЕДНИЕ МАКСИМУМЫ КАЖДОГО ЖЕСТА', maxmass)


ЖЕСТЫ
r_ladon_d.txt
r_kulak_d.txt
l_ladon_d2.txt
thrashhhh3.txt
l_kulak_d.txt

СРЕДНИЕ МАКСИМУМЫ КАЖДОГО ЖЕСТА [8.393065163642541, 9.219261257684751, 7.70850971501538, 5.763714134812621, 12.928185076178433]


In [2]:
# Подключаем камеру

#stdoutOrigin=sys.stdout 
#sys.stdout = open("log.txt", "w") #ДВЕ СТРОЧКИ ОТВЕЧАЮТ ЗА НАЧАЛО ВВОДА В log.txt

cap = cv2.VideoCapture(0) 
cap.set(3, 640) # Width
cap.set(4, 480) # Lenght
cap.set(10, 100) # Brightness

tempg = os.listdir(path_to_dataset) 
gests = [i[:-4] for i in tempg]

mpHands = mp.solutions.hands
hands = mpHands.Hands(False)
npDraw = mp.solutions.drawing_utils
a = []
pTime = 0
cTime = 0
data = []
tt = 0
set_play_pos=True
check=True
check_video=True
cnt=0
while True: #ВЕСЬ ЦИКЛ ПРОХОДИТ КАЖДЫЙ КАДР
    if tt == 1000: #ЛИМИТ НА КОЛИЧЕСТВО КАДРОВ
        break
    cnt+=1
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    cx8=0
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    pred2 = []
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            b = []
            for id, lm in enumerate(handLms.landmark):
                h,w,c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                b.append(lm.x)
                b.append(lm.y)
                if id == 8:
                    cx8 = cx
                if  id == 8 or id == 12:
                    cv2.circle(img, (cx,cy),10,(255,0,255),cv2.FILLED)
            flag = 0
            if len(b) == 42:
                pred = net(torch.tensor(b))
                pred2 = pred.detach().numpy()
            else:
                flag = 1
            npDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            a. append(b)
    tt+=1
    ans = 'nogest'
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    exist = [float(i) for i in pred2]
    if len(exist) == len(gests):
        tempans = max(exist)
        tempansid = exist.index(tempans)
        if flag ==0:    
            if tempans >= maxmass[tempansid] - (maxmass[tempansid])/5:
                if check==True:
                    match tempansid:
                        case 0:
                                webbrowser.open("https://www.youtube.com/watch?v=dQw4w9WgXcQ&ab_channel=RickAstley")
                                print('r_ladon')
                                ans='r_ladon'
                                check_video=False
                                check=False
                        case 1:
                                webbrowser.open("https://www.youtube.com/watch?v=A1Qb4zfurA8&ab_channel=Enjoykipip")
                                print('r_kulak')
                                ans='r_kulak'
                                check_video=False
                                check=False
                        case 2:
                                print('l_ladon')
                                ans='l_ladon'
                                if set_play_pos==True:
                                    keyboard.send("space")
                                    set_play_pos=False
                                check=False
                        case 3:
                                print('thrash')
                                ans='thrash'
                        case 4:
                                print('l_kulak')
                                ans='l_kulak'
                                if set_play_pos==False:
                                    keyboard.send("space");
                                    set_play_pos=True
                                check=False
                else:
                     if check_video==True:
                        if cnt>=5:
                            check=True
                            cnt=0
                     else:
                          if cnt>=30:
                            check=True
                            check_video=True
                            cnt=0
            else:
                ans = 'nogest'
    
    cv2.putText(img, str(ans),(10,30), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0), 2) # ФреймРейт
    #cv2.putText(img, str(cx8),(100,30), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0), 2)
    cv2.imshow('python', img)
    if cv2.waitKey(20) == 27: # exit on ESC
        break

cv2.destroyWindow("python")
cap.release()
cv2.waitKey(1)
#print(a) # a is massive of data we need
#sys.stdout.close()
#sys.stdout=stdoutOrigin #ДВЕ СТРОЧКИ ЗА ЗАКРЫТИЕ log.txt


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


thrash
r_ladon
r_ladon
r_ladon
r_ladon
thrash
thrash
thrash
l_ladon
l_ladon
l_ladon
l_kulak
l_kulak
l_ladon
l_ladon
l_kulak
l_kulak
thrash
thrash
l_ladon
l_ladon
thrash
thrash
thrash
thrash


ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x105cfa980>>)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/iostream.py", line 604, in _flush
    self.session.send(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/jupyter_client/session.py", line 862, in send
    msg["tracker"] = tracker
                     ^^^^^^^
UnboundLocalError: cannot access local variable 'tracker' where it is not associated with a value


NameError: name 'stdoutOrigin' is not defined